# Sklearn algorithms

We attempt to use a few algorithm in sklearn to perform word segmentation on Khmer document as a based comparison to CRF.

With one hot encoding, we a few algorithm like Random Forest and Linear Regression get to 93%. While Naive Bayes is around 89%.

## Environment Setup

### Downloading Data
This section shows how to download and extract data to be analyzed. We save our data in github in a compressed format. There are multiple sizes depends on which one you want to analyze. We started with 100 articles and we run through the code. This makes it fast to run the experiment as we fine tune the parameters.

In [0]:
size = ["100","500","1000","5000","10000"]
# Select available size (0-4) that you want to use
docsize = size[0]
data_dir = "kh_data_" + docsize
file_name = data_dir + ".zip"

In [0]:
# Download the file from `url` and save it
import urllib.request
base_url = "https://github.com/phylypo/segmentation-crf-khmer/raw/master/data/"
url = base_url + file_name
print("Downloading from:", url)
urllib.request.urlretrieve(url, file_name)

('kh_data_100.zip', <http.client.HTTPMessage at 0x7f125ab546d8>)

In [0]:
# Run shell command to extract files

# remove previous existing directory if rerun
!rm -rf {data_dir}
print("- Unzipping the file and show last few extracted files:")
!unzip {file_name} | tail -10

print("- Count the number of files:")
!ls -al {data_dir}/*_seg.txt | wc -l

- Unzipping the file and show last few extracted files:
  inflating: kh_data_100/313540_seg.txt  
  inflating: kh_data_100/313541_orig.txt  
  inflating: kh_data_100/313541_seg.txt  
  inflating: kh_data_100/313544_orig.txt  
  inflating: kh_data_100/313544_seg.txt  
  inflating: kh_data_100/313545_orig.txt  
  inflating: kh_data_100/313545_seg.txt  
  inflating: kh_data_100/313546_orig.txt  
  inflating: kh_data_100/313546_seg.txt  
  inflating: kh_data_100/meta.txt    
- Count the number of files:
100


## Read data files

Each files contain mulitple lines of text. Each line may end with a Khmer sentence terminator character.

We read all text into variable ***seg_text*** as a list of list string. seg_text[0] is the content of first file. 

Eg. First line of the first file would be seg_text[0][0].

Similarly we have orig_text which read the orignal article format. This is not used for the training. This can be use to compare the orignal text with the segmented text.

In [0]:
import glob
import re

path = data_dir + '/*_seg.txt'   
files=glob.glob(path)

# global variables that use through out
seg_text = []
orig_text = []
# unique id of the article that can be matched to docId in meta.txt
doc_ids = []
for file in files:
  filenum = re.search(r'\d+_', file).group(0)
  doc_ids.append(filenum.replace("_",""))
  f=open(file, 'r')
  lines = f.readlines()
  f.close()
  seg_text.append(lines)
  
  # limit to 9K to avoid memory issue
  if len(seg_text) >= 9000: break #kcc=9000, char:5000, crash char on 7000
  
  # read orig text -- comment out (10K docs which do not have orig text)
  #f=open(file.replace('_seg.txt', '_orig.txt'), 'r')
  #lines = f.readlines()
  #f.close()
  #orig_text.append(lines)
  
print("number of file:", len(files))
print("number of doc_ids:", len(doc_ids)) # can see the origial text
print("len of seg_text:", len(seg_text), ' sample data[0][0]:', seg_text[0][0])
#print("len of orig_text:", len(orig_text), ' sample data[0][0]:', orig_text[0][0])

number of file: 100
number of doc_ids: 100
len of seg_text: 100  sample data[0][0]:  អ្នក បាន ទៅ  ដល់ តំបន់ អភិវឌ្ឍន៍ របស់  ឧកញ៉ា  ម៉ុង ឫទ្ធី  នាំគ្នា សរសើរ មិន ដាច់ ពី មាត់  ព្រោះ បាន ឃើញ ទេសភាព ដ៏ ស្រស់ ត្រកាល  មាន ព្រៃឈើ  ធម្មជាតិ  រាប់ លាន ដើម  ហើយ បាន ទិញ យក លំនៅឋាន សម្រាប់ មក លំហែ កាយ ខ្លះ ទិញ បី ខ្នង  ខ្លះ ទៀត ទិញ ពីរ ខ្នង …



## Prepare Data
Each  seg text contains list of sentences. We want a list of sentences in variable 'sentences'. We also clean up the extra spaces or new line character.

Here we allow to fix any known bad segmented data such as names.

In [0]:
#@title Parse and correct sentences
def correct_str(str):
  # fix some bad segmented data - especially proper noun
  str = str.replace('ស៊ុន ចាន់ ថុល', 'ស៊ុន ចាន់ថុល')
  str = str.replace('សាយ មក រា','សាយ មករា')
  str = str.replace('រោង កុន','រោងកុន')
  str = str.replace('ដេី មជ្រៃ ','ដេីមជ្រៃ ')
  str = str.replace('ដេី ម ','ដេីម ')
  str = str.replace('មន្ទី ពេទ្យ ','មន្ទីពេទ្យ ')
  str = str.replace('ជាដរាបត ទៅ ','ជា ដរាប តទៅ ')
  str = str.replace('ដេី ម្បី ','ដេីម្បី ')
  str = str.replace('ធ្វេី ការកាត់ យក','ធ្វេីការ កាត់ យក')
  str = str.replace('ភូមិ ផ្សារ ដី ហុយ','ភូមិ ផ្សារដីហុយ')
  str = str.replace('សុខ ពេញ វុធ ','សុខ ពេញវុធ ')
  str = str.replace('ប្រតិបត្តិការបង្ក្រាប','ប្រតិបត្តិការ បង្ក្រាប')
  str = str.replace('health . com . kh','health.com.kh')
  str = str.replace('ពួយ ច័ន្ទ សុគុណ ','ពួយ ច័ន្ទសុគុណ ')
  str = str.replace('ភូមិ ថ្ម ដា ','ភូមិ ថ្មដា ')
  str = str.replace('សង្កាត់ បឹង កន្សែង','សង្កាត់ បឹងកន្សែង')
  str = str.replace('ក្រុង បាន លុង','ក្រុង បានលុង')
  str = str.replace('នុត សុគន្ធ ផាន់ នី ','នុត សុគន្ធ ផាន់នី ')
  str = str.replace('ព្រះរាជាណា ច្រក កម្ពុជា','ព្រះរាជាណាច្រ ក កម្ពុជា')
  str = str.replace('កាក បាត ក្រហម','កាកបាត ក្រហម')
  str = str.replace('ស្រុក កំពង់ត្រា ច ','ស្រុក កំពង់ត្រាច ')
  str = str.replace('ឃុំ ថ្ម កែវ','ឃុំ ថ្មកែវ')
  str = str.replace('ស្រុក អង្គរ ជ័យ','ស្រុក អង្គរជ័យ')
  str = str.replace('កោះ តា កូវ ឃុំ ឬ ស្សី','កោះ តាកូវ ឃុំ ឬស្សី')
  str = str.replace('ម៉ាក សែន សូនីតា ','ម៉ាក សែនសូនីតា ')
  str = str.replace('ជា ដេី ម នោះ','ជា ដេីម នោះ')
  str = str.replace('ដេី ម្បី ជ្រាប','ដេីម្បី ជ្រាប')
  str = str.replace('ក រករណី','ករ ករណី')
  str = str.replace('ទោ ចក្រ យានយន្ត','ទោចក្រយានយន្ត')
  str = str.replace('ឃុំ អន្លង់ វិល ','ឃុំ អន្លង់វិល ')
  str = str.replace(' ពោធ៍ សាត់ ',' ពោធ៍សាត់ ')
  str = str.replace('ណាក់ ស្រី ណា','ណាក់ ស្រីណា')
  str = str.replace('ពិធីការ នីមួយ','ពិធីការនី មួយ')
  str = str.replace('តូរុន តូ ','តូរុនតូ ')
  str = str.replace('ដាយ ណូស័រ ','ដាយណូស័រ ')
  str = str.replace('ហ្គារ៉ាល់ ហូស ','ហ្គារ៉ាល់ហូស ')
  str = str.replace('ខេមរៈ សិរី មន្ត','ខេមរៈ សិរីមន្ត')
  str = str.replace('ឱម យ៉ិន ទៀង','ឱម យ៉ិនទៀង')
  str = str.replace('មហា លាប','មហាលាប')
  str = str.replace('ការចាប់អារម្ម ណ៏ ','ការចាប់អារម្មណ៏ ')
  str = str.replace('វ៉ិច ទ័រ ','វ៉ិចទ័រ ')
  str = str.replace('សុវណ្ណ ឬ ទ្ធី ','សុវណ្ណ ឬទ្ធី ')
  str = str.replace('ខាន់ ចាន់ សុផល','ខាន់ ចាន់សុផល')
  str = str.replace('ទំនួលខុសត្រូវ ','ទំនួល ខុសត្រូវ ')
  str = str.replace('ឯ ណេះ','ឯណេះ') #confirm with Chuon Nath Dict
  str = str.replace('ហេយ៍ វ៉ិន','ហេយ៍វ៉ិន') #heaven
  str = str.replace('សុខ ពេញ វុធ','សុខ ពេញ វុធ')
  str = str.replace('វ៉ុល កា','វ៉ុលកា')
  str = str.replace('ជា ម ហុី ម ','ជាម ហុីម ')
  str = str.replace(' ការ ប្រកាស ',' ការ ប្រកាស ')
  str = str.replace('អនុ ប្រធាន','អនុប្រធាន')
  str = str.replace('កិច្ចខិតខំប្រឹងប្រែង','កិច្ច ខិតខំ ប្រឹងប្រែង')
  str = str.replace('សស្អាត','ស ស្អាត')
  str = str.replace('ហាន់ ជី អ៊ុន ','ហាន់ ជីអ៊ុន ')
  str = str.replace('ស៊ុន ចាន់ ថុល','ស៊ុន ចាន់ថុល')
  str = str.replace('លីន ដា ','លីនដា ')
  str = str.replace('យានយន្តបន្ទាប់ពី','យានយន្ត បន្ទាប់ពី')
  str = str.replace('ព្រៃ ស្អាក ស្រុក អន្លង់ វែង','ព្រៃស្អាក ស្រុក អន្លង់វែង')
  str = str.replace(' ប៉ា កុង ',' ប៉ាកុង ') # first 50 articles
  str = str.replace('ឈើ អុស ក្រាក់','ឈើ អុសក្រាក់')
  str = str.replace('រតនៈ គីរី ','រតនៈគីរី ')
  str = str.replace('ឯក ឧត្តម','ឯកឧត្តម')
  str = str.replace(' ឱម យិន ទៀង',' ឱម យិនទៀង')
  str = str.replace('ឱម យិន ទៀង','ឱម យិនទៀង')
  str = str.replace('រៀប ចំដែនដី ','រៀបចំ ដែនដី ') # orig has space after រៀប
  str = str.replace(' ឱវា ទ ',' ឱវាទ ') # correct spelling
  str = str.replace('អ៊ិន វេស មិន ','អ៊ិនវេសមិន ')
  str = str.replace('សហគម ន៏ ','សហគមន៏ ')
  str = str.replace(' គ្រូ សារ ',' គ្រូសារ ') # orig has bad u\200b but should be removed before seg
  str = str.replace('សុផុនពេល សួរ','សុផុន ពេល សួរ')
  str = str.replace('ស្រុក កែវ សីម៉ា','ស្រុក កែវសីម៉ា')
  str = str.replace(' ហាន់ ជ័យ ',' ហាន់ជ័យ ')
  str = str.replace(' បូរី ស ចនសុន ',' បូរីស ចនសុន ')
  str = str.replace('បូរី ស ចន សុន ','បូរីស ចនសុន ')
  str = str.replace('អំណ រ គុណ','អំណរ គុណ') # bad orig
  str = str.replace('ផលប៉ះពាល់ ','ផល ប៉ះពាល់ ')
  str = str.replace('អ្នកវិនិយោគ ','អ្នក វិនិយោគ ')
  str = str.replace('ភូមិ ស្រះ ជ្រៃ ','ភូមិ ស្រះជ្រៃ ')
  str = str.replace('ឃុំ បន្ទាយ ឆ្មា ','ឃុំ បន្ទាយឆ្មា ')
  str = str.replace('ស្រុក ថ្ម ពួក ','ស្រុក ថ្មពួក ')
  str = str.replace(' ទ្វីត ធឺ ',' ទ្វីតធឺ ')
  str = str.replace(' ធំ ដុំ ',' ធំដុំ ')
  str = str.replace(' រួមមាន ',' រួម មាន ')
  str = str.replace(' គុយ វ៉ែត ',' គុយវ៉ែត ')
  str = str.replace('ងងុយគេង ','ងងុយ គេង ')
  str = str.replace(' ស ម រង្ស៊ី ',' សម រង្ស៊ី ')
  str = str.replace('លោក សោ ចាន់ ដេត ','លោក សោ ចាន់ដេត ')
  str = str.replace('ជាលាយលក្ខណ៍អក្សរ ','ជា លាយ លក្ខណ៍ អក្សរ ')
  str = str.replace('ផលិតកម្មវិធី ','ផលិតកម្ម វិធី ')
  str = str.replace(' បូរីស ចន សុន ',' បូរីស ចនសុន ')
  str = str.replace(' សាជាថ្មី ',' សា ជា ថ្មី ')
  str = str.replace('ឧកញ៉ា ស្រី ចាន់ ថន ','ឧកញ៉ា ស្រី ចាន់ថន ')
  str = str.replace('ភូមិ ព្រៃ ល្វា ','ភូមិ ព្រៃល្វា ')
  str = str.replace('សង្កាត់ ចោម ចៅ ','សង្កាត់ ចោមចៅ ')
  str = str.replace(' ខណ្ឌ ពោធិ៍ សែន ',' ខណ្ឌ ពោធិ៍សែន ')
  str = str.replace('ឃុំ ព្រែក តា មាក់ ','ឃុំ ព្រែកតាមាក់ ')
  str = str.replace(' ទៅ បាត់បង្ក ',' ទៅ បាត់ បង្ក ')
  str = str.replace(' កើតមានឡើង ',' កើត មាន ឡើង ')
  str = str.replace(' លូក លាន់ ',' លូកលាន់ ')
  str = str.replace('លោក ជិន ម៉ាលី ន ','លោក ជិន ម៉ាលីន ')
  str = str.replace(' អ្នករាយការណ៍ ',' អ្នក រាយការណ៍ ')
  str = str.replace(' គង់ រ៉ៃ យ៉ា ',' គង់ រ៉ៃយ៉ា ')
  str = str.replace('លោក សួង នាគ ព័ន្ធ ','លោក សួង នាគព័ន្ធ ')
  str = str.replace(' ធ្វើទុក្ខបុកម្នេញ ',' ធ្វើ ទុក្ខបុកម្នេញ ')  # doc 75 mark
  str = str.replace('បោះបង់ចោល ','បោះបង់ ចោល ')
  str = str.replace(' ខ្លួនឯង ',' ខ្លួន ឯង ')
  str = str.replace('សប្បាយចិត្ត','សប្បាយ ចិត្ត')
  str = str.replace('មើលទៅ ','មើល ទៅ ')
  str = str.replace('សោយសុខ ','សោយ សុខ ')
  str = str.replace(' ទៅលើ ',' ទៅ លើ ')
  str = str.replace(' ឃុំខ្លួន ',' ឃុំ ខ្លួន ')
  str = str.replace('ជាប់ឃុំ ','ជាប់ ឃុំ ')
  str = str.replace('លោក ហេង ដូន នី ','លោក ហេង ដូននី ')
  str = str.replace('លោក កែម គិម ស្រន់ ','លោក កែម គិមស្រន់ ')
  str = str.replace(' ស្រុក វាល វែង ',' ស្រុក វាលវែង ')
  str = str.replace('ចោទប្រកាន់តែ ','ចោទប្រកាន់ តែ ') # doc 80 mark
  str = str.replace(' ឈិញ ស៊ី ថា ',' ឈិញ ស៊ីថា ')
  str = str.replace(' ដាក់ពាក្យបណ្តឹង ',' ដាក់ ពាក្យ បណ្តឹង ')
  str = str.replace('ម៉ាក ណូ គា ','ម៉ាក ណូគា ')
  str = str.replace(' ធូ ស្រី ទូច ',' ធូ ស្រីទូច ')
  str = str.replace(' ម៉ូតូកង់ បី',' ម៉ូតូ កង់ បី')
  str = str.replace('ឃុំ មង់ រៀវ ','ឃុំ មង់រៀវ ')
  str = str.replace('លោក ជា ចាន់ តូ ','លោក ជា ចាន់តូ ')
  str = str.replace(' ស៊ែ ស្វី ច ',' ស៊ែស្វីច ')
  str = str.replace(' ប្រតិបត្តិការផ្ទេរ ',' ប្រតិបត្តិការ ផ្ទេរ ')
  str = str.replace(' បញ្ជាការស្រាល ',' បញ្ជាការ ស្រាល')
  str = str.replace(' F - 16 ',' F-16 ')
  str = str.replace(' C - 17 ',' C-17 ')
  str = str.replace(' គោលនយោបាយ ',' គោល នយោបាយ ') # id 90 mark
  str = str.replace(' ហាន សុខ ន ',' ហាន សុខន ')
  str = str.replace(' ខៀវ ទេ ព រង្ស៊ី កាញារី ទ្ធ ',' ខៀវ ទេព រង្ស៊ី កាញារីទ្ធ ')
  str = str.replace(' ជី ហែ ',' ជីហែ ')
  str = str.replace('វិទ្យាល័យ ជី ហែរ ','វិទ្យាល័យ ជីហែរ ')
  str = str.replace('ស្រុក កោះ សូទិន ','ស្រុក កោះសូទិន ')
  str = str.replace(' ជំរុញឲ្យ ',' ជំរុញ ឲ្យ ')
  str = str.replace(' ការបាក់ទឹកចិត្ត ',' ការបាក់ ទឹកចិត្ត ')
  str = str.replace(' ហង់ ជួន ណារីតា ',' ហង់ ជួនណារីតា ')
  str = str.replace(' ជាទីមោទនៈ ',' ជា ទី មោទនៈ ')
  str = str.replace(' សឿ សុជា តា ',' សឿ សុជាតា ')
  str = str.replace(' ហង់ ជួន ណារ៉ុន ',' ហង់ ជួនណារ៉ុន ')
  str = str.replace(' ក សាង ធនធាន ',' កសាង ធនធាន ')
  str = str.replace('ស្នង ឬ ស្សី ','ស្នង ឬស្សី ')
  str = str.replace('លោក ខៀវ កាញារី ទ្ធ ','លោក ខៀវ កាញារីទ្ធ ')
  str = str.replace(' ត្រេន ដី ង ',' ត្រេនដីង ') #trending
  str = str.replace(' ទៅវិញទៅមក ',' ទៅ វិញ ទៅ មក ')
  str = str.replace(' ក្ដី ស្រមៃ ',' ក្ដីស្រមៃ ')
  str = str.replace(' ឬ ស្សីកែវ ',' ឬស្សីកែវ ')
  str = str.replace(' អេង សុវណ្ណ តារា ',' អេង សុវណ្ណតារា ')
  str = str.replace(' កុង ទីន រ័',' កុងទីនរ័')
  str = str.replace('ជាបន្តបន្ទាប់ ','ជា បន្តបន្ទាប់ ')
  str = str.replace('ចលត័ជាប្រចាំ ','ចលត័ ជា ប្រចាំ ')
  str = str.replace(' ឃុំ ដី ឥដ្ឋ ',' ឃុំ ដីឥដ្ឋ ')
  str = str.replace(' វី ដែ អូ ',' វីដែអូ ') #misspell វីដេអូ 
  str = str.replace(' ត្រី ខ ទឹក ត្រី ',' ត្រីខ ទឹកត្រី ')
  str = str.replace(' បង្ខំឲ្យ ',' បង្ខំ ឲ្យ ')
  return str

def cleanup_str(str):
  #str = correct_str(str)
  # remove special characters
  str = str.replace(u"\u2028", "") # line separator
  str = str.replace(u"\u200a", "")  # hair space
  return str.strip().replace('\n','').replace('  ',' ')

# setup training data using seg
sentences = list()
for i, text in enumerate(seg_text):
    for sentence in text:
        sentences.append(cleanup_str(sentence))
print("total len of sentences:", len(sentences))

# id=99
# print("id:",doc_ids[id])
# for i,s in enumerate(seg_text[id]):
#   print("sample sentence: %s" %(cleanup_str(s)))
#   print("orig sentence: %s" %(repr(orig_text[id][i])))

total len of sentences: 1028


## Create Features
This section implement functions to generate KCC and features for CRF.

### KCC

We implementation the segmentation of Khmer orthographic or Khmer character cluster (KCC) in a simpler way that the rule illustrate by the paper (X). Instead of check for all possible rules from the start of each character, we just check the next character and determine whether it is a type that  start a new KCC. If so, the current character is the end of the current KCC. If there are series of invalid KCC string, this approach does not separate those. But segmented it would leave the next segment invalid. (***to make clearer***)


In [0]:
#@title Segment into KCC
# list of constants needed for KCC and feature generation
# consonant and independent vowels
KHCONST = set(u'កខគឃងចឆជឈញដឋឌឍណតថទធនបផពភមយរលវឝឞសហឡអឣឤឥឦឧឨឩឪឫឬឭឮឯឰឱឲឳ')
KHVOWEL = set(u'឴឵ាិីឹឺុូួើឿៀេែៃោៅ\u17c6\u17c7\u17c8')
# subscript, diacritics
KHSUB = set(u'្')
KHDIAC = set(u"\u17c9\u17ca\u17cb\u17cc\u17cd\u17ce\u17cf\u17d0") #MUUSIKATOAN, TRIISAP, BANTOC,ROBAT,
KHSYM = set('៕។៛ៗ៚៙៘')
KHNUMBER = set(u'០១២៣៤៥៦៧៨៩0123456789') # remove 0123456789
# lunar date:  U+19E0 to U+19FF ᧠...᧿
KHLUNAR = set('᧠᧡᧢᧣᧤᧥᧦᧧᧨᧩᧪᧫᧬᧭᧮᧯᧰᧱᧲᧳᧴᧵᧶᧷᧸᧹᧺᧻᧼᧽᧾᧿')

def is_khmer_char(ch):
  if (ch >= '\u1780') and (ch <= '\u17ff'): return True
  if ch in KHLUNAR: return True
  return False

def is_start_of_kcc(ch):
  if is_khmer_char(ch):
    if ch in KHCONST: return True
    if ch in KHSYM: return True
    if ch in KHNUMBER: return True
    if ch in KHLUNAR: return True
    return False
  return True

# kcc base
def seg_kcc(str_sentence):
    str_sentence = str_sentence.replace(u'\u200b','')
    segs = []
    cur = ""
    for phr in str_sentence.split(' '):
        #print("PHR:[%s] len:%d" %(phr, len(phr)))
        for i,c in enumerate(phr):
            #print(i," c:", c)
            cur += c
            nextchar = phr[i+1] if (i+1 < len(phr)) else ""
            
            # cluster non-khmer chars together
            if not is_khmer_char(c) and nextchar != "" and not is_khmer_char(nextchar): continue
            # cluster number together
            if c in KHNUMBER and nextchar in KHNUMBER: continue
              
            # cluster non-khmer together
            # non-khmer character has no cluster
            if not is_khmer_char(c) or nextchar=="":
                segs.append(cur)
                cur=""
            elif is_start_of_kcc(nextchar) and not (c in KHSUB):
                segs.append(cur)
                cur=""          
    return segs

# testing some text
t1 = "យោងតាមប្រភពព័ត៌មានបានឱ្យដឹងថា កាលពីពេលថ្មីៗនេះក្រុមចក្រភពអង់គ្លេស Royal Marines ដែលមានមូលដ្ឋាននៅ Gibraltar បានរឹបអូសយកនាវាដឹកប្រេងឆៅរបស់អ៊ីរ៉ង់ដែលធ្វើដំណើរទៅកាន់រោងចក្រចម្រាញ់ប្រេងនៅក្នុងប្រទេសស៊ីរី ដោយក្រុងឡុងដ៍អះអាងថា ការរឹបអូសត្រូវបានគេសំដៅអនុវត្ត៕"
t2 = "This is a test."
t3 = "នៅរសៀលថ្ងៃទី២២ ខែ កក្កដា ឆ្នាំ២០១៩ ឯកឧត្តម គួច ចំរើន អភិបាលខេត្តព្រះសីហនុ"
print("kcc:", seg_kcc(t1))


kcc: ['យោ', 'ង', 'តា', 'ម', 'ប្រ', 'ភ', 'ព', 'ព័', 'ត៌', 'មា', 'ន', 'បា', 'ន', 'ឱ្យ', 'ដឹ', 'ង', 'ថា', 'កា', 'ល', 'ពី', 'ពេ', 'ល', 'ថ្មី', 'ៗ', 'នេះ', 'ក្រុ', 'ម', 'ច', 'ក្រ', 'ភ', 'ព', 'អ', 'ង់', 'គ្លេ', 'ស', 'Royal', 'Marines', 'ដែ', 'ល', 'មា', 'ន', 'មូ', 'ល', 'ដ្ឋា', 'ន', 'នៅ', 'Gibraltar', 'បា', 'ន', 'រឹ', 'ប', 'អូ', 'ស', 'យ', 'ក', 'នា', 'វា', 'ដឹ', 'ក', 'ប្រេ', 'ង', 'ឆៅ', 'រ', 'ប', 'ស់', 'អ៊ី', 'រ៉', 'ង់', 'ដែ', 'ល', 'ធ្វើ', 'ដំ', 'ណើ', 'រ', 'ទៅ', 'កា', 'ន់', 'រោ', 'ង', 'ច', 'ក្រ', 'ច', 'ម្រា', 'ញ់', 'ប្រេ', 'ង', 'នៅ', 'ក្នុ', 'ង', 'ប្រ', 'ទេ', 'ស', 'ស៊ី', 'រី', 'ដោ', 'យ', 'ក្រុ', 'ង', 'ឡុ', 'ង', 'ដ៍', 'អះ', 'អា', 'ង', 'ថា', 'កា', 'រ', 'រឹ', 'ប', 'អូ', 'ស', 'ត្រូ', 'វ', 'បា', 'ន', 'គេ', 'សំ', 'ដៅ', 'អ', 'នុ', 'វ', 'ត្ត', '៕']


In [0]:
# generate list of (word, label), not splitting into phrases, just remove spaces
def gen_kcc_with_label(sentence):
    words = sentence.split()
    final_kccs = []
    for word in words:
        kccs = seg_kcc(word)
        labels = [1 if (i==0) else 0 for i, k in enumerate(kccs)]
        final_kccs.extend(list(zip(kccs,labels)))
    return final_kccs

# test label
ts = "This is a test"
ts = "នៅ រសៀល ថ្ងៃ ទី ២២ ខែ កក្កដា ឆ្នាំ ២០១៩ ។"
kccs = seg_kcc(ts)
kl = gen_kcc_with_label(ts)
print("len of kcc:", len(kccs), " data:", kccs)
print("kcc with label len:", len(kl), " data:", kl)

len of kcc: 14  data: ['នៅ', 'រ', 'សៀ', 'ល', 'ថ្ងៃ', 'ទី', '២២', 'ខែ', 'ក', 'ក្ក', 'ដា', 'ឆ្នាំ', '២០១៩', '។']
kcc with label len: 14  data: [('នៅ', 1), ('រ', 1), ('សៀ', 0), ('ល', 0), ('ថ្ងៃ', 1), ('ទី', 1), ('២២', 1), ('ខែ', 1), ('ក', 1), ('ក្ក', 0), ('ដា', 0), ('ឆ្នាំ', 1), ('២០១៩', 1), ('។', 1)]


In [0]:
EN = set(u'abcdefghijklmnopqrstuvwxyz0123456789')

# E=English, C=Consonant, W=wowel, N=number, O=Other, S=subcript, D=Diacritic, NS=no_space
# roll up to: NS, C, W, S, D
NS = 'NS'
def get_type(chr):
  if chr.lower() in EN: return "NS"
  if chr in KHCONST: return "C"
  if chr in KHVOWEL: return "W"
  if chr in KHNUMBER: return "NS"
  if chr in KHSUB: return "S"
  if chr in KHDIAC: return "D"
  return "NS" #not U - same as N

# non-khmer character that we should not separate like number
# multiple characters are false
def is_no_space(k):
  if len(k) > 1: return False
  if get_type(k)==NS: return True
  return False

def kcc_type(k):
  if len(k)==1: return get_type(k)
  else: return "K" + str(len(k))

print("kcc_type for u'\u17cb'", kcc_type(u'\u17cb'))
print("kcc_type for u'A'", kcc_type('A'))
print("kcc_type for u'២'", kcc_type('២'))
print("kcc_type for 'ថ្ងៃ'", kcc_type('ថ្ងៃ'))
print("is NS for 'A':", is_no_space("A"))
print("is NS for 'ថ្ងៃ':", is_no_space('ថ្ងៃ'))

kcc_type for u'់' D
kcc_type for u'A' NS
kcc_type for u'២' NS
kcc_type for 'ថ្ងៃ' K4
is NS for 'A': True
is NS for 'ថ្ងៃ': False


In [0]:
#@title Deinfe CRF features
# only pass in kccs list (without labels)
def kcc_to_features(kccs, i):
    maxi = len(kccs)
    kcc = kccs[i]

    features = {
        'bias': 1.0,
        'kcc': kcc,
        't': kcc_type(kcc),
        'ns': is_no_space(kcc)
    }
    if i >= 1:
        features.update({
            'kcc[-1]'  : kccs[i-1],
            'kcc[-1]t' : kcc_type(kccs[i-1]),
            'kcc[-1:0]': kccs[i-1] + kccs[i],
            'ns-1' : is_no_space(kccs[i-1])
        })
    else:
        features['BOS'] = True

    if i >= 2:
        features.update({
            'kcc[-2]'   : kccs[i-2],
            'kcc[-2]t'  : kcc_type(kccs[i-2]),
            'kcc[-2:-1]': kccs[i-2] + kccs[i-1],
            'kcc[-2:0]' : kccs[i-2] + kccs[i-1] + kccs[i],
        })
    if i >= 3:
        features.update({
            'kcc[-3]'   : kccs[i-3],
            'kcc[-3]t'  : kcc_type(kccs[i-3]),
            'kcc[-3:0]' : kccs[i-3] + kccs[i-2] + kccs[i-1] + kccs[i],
            'kcc[-3:-1]': kccs[i-3] + kccs[i-2] + kccs[i-1],
            'kcc[-3:-2]': kccs[i-3] + kccs[i-2],
        })

    if i < maxi-1:
        features.update({
            'kcc[+1]'  : kccs[i+1],
            'kcc[+1]t'  : kcc_type(kccs[i+1]),
            'kcc[+1:0]': kccs[i] + kccs[i+1],
            'ns+1' : is_no_space(kccs[i+1])

        })
    else:
        features['EOS'] = True

    if i < maxi-2:
        features.update({
            'kcc[+2]'   : kccs[i+2],
            'kcc[+2]t'   : kcc_type(kccs[i+2]),
            'kcc[+1:+2]': kccs[i+1] + kccs[i+2],
            'kcc[0:+2]' : kccs[i+0] + kccs[i+1] + kccs[i+2],
            'ns+2' : is_no_space(kccs[i+2])
        })
    if i < maxi-3:
        features.update({
            'kcc[+3]'   : kccs[i+3],
            'kcc[+3]t'   : kcc_type(kccs[i+3]),
            'kcc[+2:+3]': kccs[i+2] + kccs[i+3],
            'kcc[+1:+3]': kccs[i+1] + kccs[i+2] + kccs[i+3],
            'kcc[0:+3]' : kccs[i+0] + kccs[i+1] + kccs[i+2] + kccs[i+3],
        })

    return features

def generate_kccs_label_per_phrase(sentence):
    phrases = sentence.split()
    print("prep_kcc_labels -- number of phrases:", len(phrases))
    final_kccs = []
    for phrase in phrases:
        kccs = seg_kcc(phrase)
        labels = [1 if (i==0) else 0 for i, k in enumerate(kccs)]
        final_kccs.extend(list(zip(kccs,labels)))
    return final_kccs

def create_kcc_features(kccs):
    return [kcc_to_features(kccs, i) for i in range(len(kccs))]

# take label in second element from kcc with label
def create_labels_from_kccs(kccs_label):
    return [str(part[1]) for part in kccs_label]

# test
ts = "នៅ រសៀល ថ្ងៃ ទី ២២ ខែ កក្កដា ឆ្នាំ ២០១៩ ។"
kccs = seg_kcc(ts)
kccs_label = gen_kcc_with_label(ts) # only need for training
print("kcc with label:", kccs_label )
print("format features:", create_kcc_features(kccs))
print("create labels:", create_labels_from_kccs(kccs_label))
print("create labels:", create_kcc_features(seg_kcc("NS123")))

kcc with label: [('នៅ', 1), ('រ', 1), ('សៀ', 0), ('ល', 0), ('ថ្ងៃ', 1), ('ទី', 1), ('២២', 1), ('ខែ', 1), ('ក', 1), ('ក្ក', 0), ('ដា', 0), ('ឆ្នាំ', 1), ('២០១៩', 1), ('។', 1)]
format features: [{'bias': 1.0, 'kcc': 'នៅ', 't': 'K2', 'ns': False, 'BOS': True, 'kcc[+1]': 'រ', 'kcc[+1]t': 'C', 'kcc[+1:0]': 'នៅរ', 'ns+1': False, 'kcc[+2]': 'សៀ', 'kcc[+2]t': 'K2', 'kcc[+1:+2]': 'រសៀ', 'kcc[0:+2]': 'នៅរសៀ', 'ns+2': False, 'kcc[+3]': 'ល', 'kcc[+3]t': 'C', 'kcc[+2:+3]': 'សៀល', 'kcc[+1:+3]': 'រសៀល', 'kcc[0:+3]': 'នៅរសៀល'}, {'bias': 1.0, 'kcc': 'រ', 't': 'C', 'ns': False, 'kcc[-1]': 'នៅ', 'kcc[-1]t': 'K2', 'kcc[-1:0]': 'នៅរ', 'ns-1': False, 'kcc[+1]': 'សៀ', 'kcc[+1]t': 'K2', 'kcc[+1:0]': 'រសៀ', 'ns+1': False, 'kcc[+2]': 'ល', 'kcc[+2]t': 'C', 'kcc[+1:+2]': 'សៀល', 'kcc[0:+2]': 'រសៀល', 'ns+2': False, 'kcc[+3]': 'ថ្ងៃ', 'kcc[+3]t': 'K4', 'kcc[+2:+3]': 'លថ្ងៃ', 'kcc[+1:+3]': 'សៀលថ្ងៃ', 'kcc[0:+3]': 'រសៀលថ្ងៃ'}, {'bias': 1.0, 'kcc': 'សៀ', 't': 'K2', 'ns': False, 'kcc[-1]': 'រ', 'kcc[-1]t': 'C', 'kcc[-1:

### Prepare data with KCC based

In [0]:
# create kccs, feature and labels for training for KCC based
kccs_label = []
kccs_only = []
labels = []
for sen in sentences:
  kcc_with_label = gen_kcc_with_label(sen)
  kccs_label.append(kcc_with_label)
  kccs_only.append(seg_kcc(sen))
  labels.append(create_labels_from_kccs(kcc_with_label))

print("len sentences:", len(sentences))
print("kccs_label:", len(kccs_label), "[0]:", kccs_label[0])
print("labels:", len(labels), "[0]:", labels[0])
print("kccs_only:", len(kccs_only), "[0]:", kccs_only[0])


len sentences: 1028
kccs_label: 1028 [0]: [('អ្ន', 1), ('ក', 0), ('បា', 1), ('ន', 0), ('ទៅ', 1), ('ដ', 1), ('ល់', 0), ('តំ', 1), ('ប', 0), ('ន់', 0), ('អ', 1), ('ភិ', 0), ('វ', 0), ('ឌ្ឍ', 0), ('ន៍', 0), ('រ', 1), ('ប', 0), ('ស់', 0), ('ឧ', 1), ('ក', 0), ('ញ៉ា', 0), ('ម៉ុ', 1), ('ង', 0), ('ឫ', 1), ('ទ្ធី', 0), ('នាំ', 1), ('គ្នា', 0), ('ស', 1), ('រ', 0), ('សើ', 0), ('រ', 0), ('មិ', 1), ('ន', 0), ('ដា', 1), ('ច់', 0), ('ពី', 1), ('មា', 1), ('ត់', 0), ('ព្រោះ', 1), ('បា', 1), ('ន', 0), ('ឃើ', 1), ('ញ', 0), ('ទេ', 1), ('ស', 0), ('ភា', 0), ('ព', 0), ('ដ៏', 1), ('ស្រ', 1), ('ស់', 0), ('ត្រ', 1), ('កា', 0), ('ល', 0), ('មា', 1), ('ន', 0), ('ព្រៃ', 1), ('ឈើ', 0), ('ធ', 1), ('ម្ម', 0), ('ជា', 0), ('តិ', 0), ('រា', 1), ('ប់', 0), ('លា', 1), ('ន', 0), ('ដើ', 1), ('ម', 0), ('ហើ', 1), ('យ', 0), ('បា', 1), ('ន', 0), ('ទិ', 1), ('ញ', 0), ('យ', 1), ('ក', 0), ('លំ', 1), ('នៅ', 0), ('ឋា', 0), ('ន', 0), ('ស', 1), ('ម្រា', 0), ('ប់', 0), ('ម', 1), ('ក', 0), ('លំ', 1), ('ហែ', 0), ('កា', 1), ('យ', 0), ('ខ្ល

In [0]:
# check for matching count -- there are cases where some speical character are not remove correctly
for i,kccs in enumerate(kccs_only):
  labels_count = len(labels[i])
  if len(kccs) != labels_count:
    print("i",i, "entry didn't match. len kccs", len(kccs), 'len labels', labels_count)
    print(kccs)
    print(labels[i])
print("No entry means we are good to go.\nIf there are entries, need to see which characters was not removed correctly.")

No entry means we are good to go.
If there are entries, need to see which characters was not removed correctly.


### Generate model inputs and split into training and test set

In [0]:
# this takes the most memory -- keep under 80% for the training to complete 4.4GB/25 => 12/25, 10K took 21.5/25.5

# create features per documents
def extract_features(kcc_line):
    return [kcc_to_features(kcc_line, i) for i in range(len(kcc_line))]

# kcc_data is list of kcc
X = [extract_features(kcc_line) for kcc_line in kccs_only]
y = labels
print("X",len(X), "X[0]", X[0])
print("y",len(y), "y[0]", y[0])


from sklearn.model_selection import train_test_split
indices = range(len(X))
X_train, X_test, y_train, y_test,X_train_idx, X_test_idx = train_test_split(X, y, indices, test_size=0.2, random_state=12) #0.20
print("len X_train:", len(X_train), " data:", X_train[0])
print("len y_train:", len(y_train), " data:", y_train[0])
print("len X_test:", len(X_test), " data:", X_test[0])
print("len y_test:", len(y_test), " data:", y_test[0])


X 1028 X[0] [{'bias': 1.0, 'kcc': 'អ្ន', 't': 'K3', 'ns': False, 'BOS': True, 'kcc[+1]': 'ក', 'kcc[+1]t': 'C', 'kcc[+1:0]': 'អ្នក', 'ns+1': False, 'kcc[+2]': 'បា', 'kcc[+2]t': 'K2', 'kcc[+1:+2]': 'កបា', 'kcc[0:+2]': 'អ្នកបា', 'ns+2': False, 'kcc[+3]': 'ន', 'kcc[+3]t': 'C', 'kcc[+2:+3]': 'បាន', 'kcc[+1:+3]': 'កបាន', 'kcc[0:+3]': 'អ្នកបាន'}, {'bias': 1.0, 'kcc': 'ក', 't': 'C', 'ns': False, 'kcc[-1]': 'អ្ន', 'kcc[-1]t': 'K3', 'kcc[-1:0]': 'អ្នក', 'ns-1': False, 'kcc[+1]': 'បា', 'kcc[+1]t': 'K2', 'kcc[+1:0]': 'កបា', 'ns+1': False, 'kcc[+2]': 'ន', 'kcc[+2]t': 'C', 'kcc[+1:+2]': 'បាន', 'kcc[0:+2]': 'កបាន', 'ns+2': False, 'kcc[+3]': 'ទៅ', 'kcc[+3]t': 'K2', 'kcc[+2:+3]': 'នទៅ', 'kcc[+1:+3]': 'បានទៅ', 'kcc[0:+3]': 'កបានទៅ'}, {'bias': 1.0, 'kcc': 'បា', 't': 'K2', 'ns': False, 'kcc[-1]': 'ក', 'kcc[-1]t': 'C', 'kcc[-1:0]': 'កបា', 'ns-1': False, 'kcc[-2]': 'អ្ន', 'kcc[-2]t': 'K3', 'kcc[-2:-1]': 'អ្នក', 'kcc[-2:0]': 'អ្នកបា', 'kcc[+1]': 'ន', 'kcc[+1]t': 'C', 'kcc[+1:0]': 'បាន', 'ns+1': False, 'kcc[+

In [0]:
#X_train, X_test, y_train, y_test,X_train_idx, X_test_idx = train_test_split(X, y, indices, test_size=0.34, random_state=12)
#9000 * 0.78 # 7k -> 2k
#9000 * 0.67 # 6k ->3k
#9000 * 0.34 # 3k ->6k
print("len y_train:", len(y_train))
print("len y_test:", len(y_test))

len y_train: 822
len y_test: 206


Our data is now format for model training. We have training date X_train and label y_train, and test set: X_test and its correspond label y_test. The test size is 20%.

Let's start the training in the next section.

### Data check - after result (colapse me)

In [0]:
print("sentences:", len(sentences))
kcc_list = [item for s in kccs_only for item in s]
print("kcc list len:", len(kcc_list))
kcc_set = set(kcc_list)
print("kcc set len", len(kcc_set), " char:", sorted(kcc_set))

#kcc_list = [item[0] for s in kccs_char_only for item in s]
#print("char list len:", len(kcc_list))
#kcc_set = set(kcc_list)
#print("char set len", len(kcc_set), " char:", sorted(kcc_set))
"""
2K: kcc list len: 73,235, char list len: 142,920
10K: kcc list len: 7,873,961

"""

sentences: 1028
kcc list len: 83606
kcc set len 1972  char: ['!', '"', '$', '%', "'t", '(', ')', ')”', '+', ',', '-', '.', '..', '...', '.1967', '/', '000281016', '010', '012', '0158', '070267070', '086', '1', '10', '10:30', '10:59', '125', '16', '1635', '17', '2,5', '20', '2000', '2004', '2019', '2020', '21', '24', '27,', '3', '4', '5', '50', '6', '6:59', '7', '7178', '7756', '7:39', '8', '824', '8857', '8961', '8:19', '8:59', '9', '911', '920', '922', '953', '9832', '9:59', ':', '=', '?', 'A', 'ABA', 'AE', 'AF', 'AFP', 'ARMOR', 'ARRIVAL', 'Admired', 'Alps', 'Amazon', 'And', 'Angeac', 'Angelina', 'Angkor', 'App', 'Apple', 'Architecture', 'Arnold', 'Asia', 'Azi', 'BBC', 'BD', 'Bachelet', 'Bangkok', 'Barack', 'Barcelona', 'Beatbox', 'Berry', 'Best', 'Bill', 'Bora', 'Borin', 'Borina', 'Boris', 'Borith', 'Branch', 'Brexit', 'Bryansk', 'Butov', 'C', 'CBR', 'CDC', 'CLICK', 'CNA', 'CSS', 'CZ', 'Cambodia', 'Cambodian', 'Cambridge', 'Catholics', 'Centre', 'Chakphop', 'Chan', 'Chann', 'Channel'

'\n2K: kcc list len: 73,235, char list len: 142,920\n10K: kcc list len: 7,873,961\n\n'

In [0]:
# show data size -- check for mismatch X and y --lenght should match
print("len of X:", len(X), X[0])
print("len of y:", len(y), y[0])
print("len of X[0]:", len(X[0]), X[0])
print("len of y[0]:", len(y[0]), y[0])
print("len train X:", len(X_train), X_train[0])
print("len train y:", len(y_train), y_train[0])
print("len test X:", len(X_test), X_test[0])
print("len test y:", len(y_test), y_test[0])

# test bad data where len X[i] not same as y[i]
for i in range(len(X)):
  if i<50000:
    if len(X[i]) != len(y[i]): 
      print(i, len(X[i]), len(y[i]))
      print(X[i])
      print(y[i])
      print(sentences[i])
      print(len(kccs_label[i]), kccs_label[i])
      print(len(kccs_only[i]), kccs_only[i])
  

len of X: 1028 [{'bias': 1.0, 'kcc': 'អ្ន', 't': 'K3', 'ns': False, 'BOS': True, 'kcc[+1]': 'ក', 'kcc[+1]t': 'C', 'kcc[+1:0]': 'អ្នក', 'ns+1': False, 'kcc[+2]': 'បា', 'kcc[+2]t': 'K2', 'kcc[+1:+2]': 'កបា', 'kcc[0:+2]': 'អ្នកបា', 'ns+2': False, 'kcc[+3]': 'ន', 'kcc[+3]t': 'C', 'kcc[+2:+3]': 'បាន', 'kcc[+1:+3]': 'កបាន', 'kcc[0:+3]': 'អ្នកបាន'}, {'bias': 1.0, 'kcc': 'ក', 't': 'C', 'ns': False, 'kcc[-1]': 'អ្ន', 'kcc[-1]t': 'K3', 'kcc[-1:0]': 'អ្នក', 'ns-1': False, 'kcc[+1]': 'បា', 'kcc[+1]t': 'K2', 'kcc[+1:0]': 'កបា', 'ns+1': False, 'kcc[+2]': 'ន', 'kcc[+2]t': 'C', 'kcc[+1:+2]': 'បាន', 'kcc[0:+2]': 'កបាន', 'ns+2': False, 'kcc[+3]': 'ទៅ', 'kcc[+3]t': 'K2', 'kcc[+2:+3]': 'នទៅ', 'kcc[+1:+3]': 'បានទៅ', 'kcc[0:+3]': 'កបានទៅ'}, {'bias': 1.0, 'kcc': 'បា', 't': 'K2', 'ns': False, 'kcc[-1]': 'ក', 'kcc[-1]t': 'C', 'kcc[-1:0]': 'កបា', 'ns-1': False, 'kcc[-2]': 'អ្ន', 'kcc[-2]t': 'K3', 'kcc[-2:-1]': 'អ្នក', 'kcc[-2:0]': 'អ្នកបា', 'kcc[+1]': 'ន', 'kcc[+1]t': 'C', 'kcc[+1:0]': 'បាន', 'ns+1': False, 'kc

## Naive Baysian and other classic ML algorithms
The idea for this section is to have have baseline of what the performance will be given the same/similar features usine in CRF. As you can see, the performances here are poor. Naive Baysian is 66% accuracy. We do see a higer number , with highest Random Forest at 89%. So some algorithm was able to learn the features but no where near our 98% accuracy. Note that there are some improvement we can do by turn all of the terms fields into one-hot-encoding so these algorithms can learn better.

We also try HMM implemention an got around 78% without much fine tune using same approach for Chinese.  Some performance on Chinese documents is around 82%.

In [0]:
#flatten X,y
print('len of y', len(y))
y_flat = [i for item in y for i in item]
X_flat = [i for item in X for i in item]
print('len of y_flat', len(y_flat))
print('len of X_flat', len(X_flat))

#Save X,y into pandas
import pandas as pd
dfnb = pd.DataFrame(X_flat)
dfnb["y"]=y_flat

# replace all NaN and convert True/False to int
dfnb["BOS"].fillna(0, inplace=True)
dfnb["BOS"].replace(True,1, inplace=True)
dfnb["EOS"].fillna(0, inplace=True)
dfnb["EOS"].replace(True,1, inplace=True)
dfnb["ns+1"].fillna(0, inplace=True)
dfnb["ns+2"].fillna(0, inplace=True)
dfnb["ns-1"].fillna(0, inplace=True)
dfnb["ns"].fillna(0, inplace=True)
dfnb["ns+1"] = dfnb["ns+1"].astype(int)
dfnb["ns+2"] = dfnb["ns+2"].astype(int)
dfnb["ns-1"] = dfnb["ns-1"].astype(int)
dfnb["ns"] = dfnb["ns"].astype(int)
dfnb.drop(columns=["bias"], inplace=True)

print(dfnb.head(20))
#print(dfnb.info())
#print(dfnb.isnull().sum(axis = 0))

len of y 1028
len of y_flat 83606
len of X_flat 83606
    kcc   t  ns  BOS kcc[+1]  ... kcc[-3:0] kcc[-3:-1]  kcc[-3:-2] EOS  y
0   អ្ន  K3   0    1       ក  ...       NaN        NaN         NaN   0  1
1     ក   C   0    0      បា  ...       NaN        NaN         NaN   0  0
2    បា  K2   0    0       ន  ...       NaN        NaN         NaN   0  1
3     ន   C   0    0      ទៅ  ...   អ្នកបាន     អ្នកបា        អ្នក   0  0
4    ទៅ  K2   0    0       ដ  ...    កបានទៅ       កបាន         កបា   0  1
5     ដ   C   0    0      ល់  ...    បានទៅដ      បានទៅ         បាន   0  1
6    ល់  K2   0    0      តំ  ...    នទៅដល់       នទៅដ         នទៅ   0  0
7    តំ  K2   0    0       ប  ...   ទៅដល់តំ      ទៅដល់         ទៅដ   0  1
8     ប   C   0    0      ន់  ...    ដល់តំប      ដល់តំ         ដល់   0  0
9    ន់  K2   0    0       អ  ...   ល់តំបន់      ល់តំប        ល់តំ   0  0
10    អ   C   0    0      ភិ  ...    តំបន់អ      តំបន់         តំប   0  1
11   ភិ  K2   0    0       វ  ...    បន់អភិ       បន់អ    

In [0]:
dfnb.columns

Index(['kcc', 't', 'ns', 'BOS', 'kcc[+1]', 'kcc[+1]t', 'kcc[+1:0]', 'ns+1',
       'kcc[+2]', 'kcc[+2]t', 'kcc[+1:+2]', 'kcc[0:+2]', 'ns+2', 'kcc[+3]',
       'kcc[+3]t', 'kcc[+2:+3]', 'kcc[+1:+3]', 'kcc[0:+3]', 'kcc[-1]',
       'kcc[-1]t', 'kcc[-1:0]', 'ns-1', 'kcc[-2]', 'kcc[-2]t', 'kcc[-2:-1]',
       'kcc[-2:0]', 'kcc[-3]', 'kcc[-3]t', 'kcc[-3:0]', 'kcc[-3:-1]',
       'kcc[-3:-2]', 'EOS', 'y'],
      dtype='object')

In [0]:
# replace 't' type mapping
types = dfnb.t.unique()
types_dict = {k: i for i,k in enumerate(types)}
dfnb['t'].replace(types_dict, inplace=True)

# create mapping of all the clusters into dictionary
kcc_terms = set(dfnb.kcc.unique())
#kcc_terms.update(dfnb['kcc[+1:+2]'].unique())
#kcc_terms.update(dfnb['kcc[+1:+3]'].unique())
#kcc_terms.update(dfnb['kcc[+1:0]'].unique())
kcc_terms.update(dfnb['kcc[+1]'].unique())
kcc_terms.update(dfnb['kcc[+1]t'].unique())
#kcc_terms.update(dfnb['kcc[+2:+3]'].unique())
kcc_terms.update(dfnb['kcc[+2]'].unique())
kcc_terms.update(dfnb['kcc[+2]t'].unique())
kcc_terms.update(dfnb['kcc[+3]'].unique())
kcc_terms.update(dfnb['kcc[+3]t'].unique())

#kcc_terms.update(dfnb['kcc[-1:0]'].unique())
kcc_terms.update(dfnb['kcc[-1]'].unique())
kcc_terms.update(dfnb['kcc[-1]t'].unique())
#kcc_terms.update(dfnb['kcc[-2:-1]'].unique())
#kcc_terms.update(dfnb['kcc[-2:0]'].unique())
kcc_terms.update(dfnb['kcc[-2]'].unique())
kcc_terms.update(dfnb['kcc[-2]t'].unique())

#kcc_terms.update(dfnb['kcc[-3:-1]'].unique())
#kcc_terms.update(dfnb['kcc[-3:-2]'].unique())
#kcc_terms.update(dfnb['kcc[-3:0]'].unique())
kcc_terms.update(dfnb['kcc[-3]'].unique())
kcc_terms.update(dfnb['kcc[-3]t'].unique())
#kcc_terms.update(dfnb['kcc[0:+2]'].unique())
#kcc_terms.update(dfnb['kcc[0:+3]'].unique())

print("count all_kcc", len(kcc_terms))
print(dfnb.head(20))

count all_kcc 1987
    kcc  t  ns  BOS kcc[+1]  ... kcc[-3:0] kcc[-3:-1]  kcc[-3:-2] EOS  y
0   អ្ន  0   0    1       ក  ...       NaN        NaN         NaN   0  1
1     ក  1   0    0      បា  ...       NaN        NaN         NaN   0  0
2    បា  2   0    0       ន  ...       NaN        NaN         NaN   0  1
3     ន  1   0    0      ទៅ  ...   អ្នកបាន     អ្នកបា        អ្នក   0  0
4    ទៅ  2   0    0       ដ  ...    កបានទៅ       កបាន         កបា   0  1
5     ដ  1   0    0      ល់  ...    បានទៅដ      បានទៅ         បាន   0  1
6    ល់  2   0    0      តំ  ...    នទៅដល់       នទៅដ         នទៅ   0  0
7    តំ  2   0    0       ប  ...   ទៅដល់តំ      ទៅដល់         ទៅដ   0  1
8     ប  1   0    0      ន់  ...    ដល់តំប      ដល់តំ         ដល់   0  0
9    ន់  2   0    0       អ  ...   ល់តំបន់      ល់តំប        ល់តំ   0  0
10    អ  1   0    0      ភិ  ...    តំបន់អ      តំបន់         តំប   0  1
11   ភិ  2   0    0       វ  ...    បន់អភិ       បន់អ         បន់   0  0
12    វ  1   0    0     ឌ្ឍ  ...

In [0]:
# this is very time consuming steps
#https://stackoverflow.com/questions/41985566/pandas-replace-dictionary-slowness
#series = series.map(lambda x: dictionary.get(x,x))

kcc_dict = {k:i for i,k in enumerate(kcc_terms)}

# replace string of characters with an associate index value
dfnb['kcc'].replace(kcc_dict, inplace=True)
#dfnb['kcc[+1:+2]'].replace(kcc_dict, inplace=True)
#dfnb['kcc[+1:+3]'].replace(kcc_dict, inplace=True)
#dfnb['kcc[+1:0]'].replace(kcc_dict, inplace=True)
dfnb['kcc[+1]'].replace(kcc_dict, inplace=True)
dfnb['kcc[+1]t'].replace(kcc_dict, inplace=True)
#dfnb['kcc[+2:+3]'].replace(kcc_dict, inplace=True)

dfnb['kcc[+2]'].replace(kcc_dict, inplace=True)
dfnb['kcc[+2]t'].replace(kcc_dict, inplace=True)
dfnb['kcc[+3]'].replace(kcc_dict, inplace=True)
dfnb['kcc[+3]t'].replace(kcc_dict, inplace=True)
#dfnb['kcc[-1:0]'].replace(kcc_dict, inplace=True)
dfnb['kcc[-1]'].replace(kcc_dict, inplace=True)
dfnb['kcc[-1]t'].replace(kcc_dict, inplace=True)
#dfnb['kcc[-2:-1]'].replace(kcc_dict, inplace=True)
#dfnb['kcc[-2:0]'].replace(kcc_dict, inplace=True)
dfnb['kcc[-2]'].replace(kcc_dict, inplace=True)
dfnb['kcc[-2]t'].replace(kcc_dict, inplace=True)

#dfnb['kcc[-3:-1]'].replace(kcc_dict, inplace=True)
#dfnb['kcc[-3:-2]'].replace(kcc_dict, inplace=True)
#dfnb['kcc[-3:0]'].replace(kcc_dict, inplace=True)
dfnb['kcc[-3]'].replace(kcc_dict, inplace=True)
dfnb['kcc[-3]t'].replace(kcc_dict, inplace=True)
#dfnb['kcc[0:+2]'].replace(kcc_dict, inplace=True)
#dfnb['kcc[0:+3]'].replace(kcc_dict, inplace=True)

print(dfnb.head(20))

     kcc  t  ns  BOS  kcc[+1]  ... kcc[-3:0] kcc[-3:-1]  kcc[-3:-2]  EOS  y
0    810  0   0    1     1378  ...       NaN        NaN         NaN    0  1
1   1378  1   0    0     1036  ...       NaN        NaN         NaN    0  0
2   1036  2   0    0     1805  ...       NaN        NaN         NaN    0  1
3   1805  1   0    0      457  ...   អ្នកបាន     អ្នកបា        អ្នក    0  0
4    457  2   0    0      219  ...    កបានទៅ       កបាន         កបា    0  1
5    219  1   0    0       49  ...    បានទៅដ      បានទៅ         បាន    0  1
6     49  2   0    0      306  ...    នទៅដល់       នទៅដ         នទៅ    0  0
7    306  2   0    0     1334  ...   ទៅដល់តំ      ទៅដល់         ទៅដ    0  1
8   1334  1   0    0      329  ...    ដល់តំប      ដល់តំ         ដល់    0  0
9    329  2   0    0      321  ...   ល់តំបន់      ល់តំប        ល់តំ    0  0
10   321  1   0    0     1193  ...    តំបន់អ      តំបន់         តំប    0  1
11  1193  2   0    0     1103  ...    បន់អភិ       បន់អ         បន់    0  0
12  1103  1 

In [0]:
# try Naive Bayes algorithm
from sklearn import model_selection, preprocessing
from sklearn import metrics, linear_model, naive_bayes, metrics, svm

#dft = dfnb[['BOS','EOS','kcc','kcc[+1:+2]','kcc[+1:+3]','kcc[+1:0]','kcc[+1]','kcc[+1]t','kcc[+2:+3]','kcc[+2]','kcc[+2]t','kcc[+3]','kcc[+3]t',
#            'kcc[-1:0]','kcc[-1]','kcc[-1]t','kcc[-2:-1]','kcc[-2:0]','kcc[-2]','kcc[-2]t',
#            'kcc[-3:-1]','kcc[-3:-2]','kcc[-3]','kcc[-3]t','kcc[0:+2]','kcc[0:+3]'
#            ]]
dft = dfnb[['BOS','EOS','kcc','kcc[+1]','kcc[+2]','kcc[+3]','kcc[-1]','kcc[-2]',
            'kcc[+1]t','kcc[+2]t','kcc[+3]t','kcc[-1]t','kcc[-2]t','kcc[-3]t'
            ]]
print(dft.head())
print(dft.columns)

# split the dataset into training and validation datasets
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(dft, dfnb['y'], test_size=0.30, random_state=1)

def train_model(classifier, trains, t_labels, valids, v_labels):
  # fit the training dataset on the classifier
  classifier.fit(trains, t_labels)

  # predict the labels on validation dataset
  predictions = classifier.predict(valids)

  return metrics.accuracy_score(predictions, v_labels)

# Naive Bayes
accuracy = train_model(naive_bayes.MultinomialNB(), train_x, train_y, valid_x, valid_y);
print("NB accuracy: ", accuracy) 
#20 featrues about 0.55
#26 all: 57
#with +1+2+3,-1-2-3,t features 0.62 accuracy

   BOS  EOS   kcc  kcc[+1]  ...  kcc[+3]t  kcc[-1]t  kcc[-2]t  kcc[-3]t
0    1    0   810     1378  ...       466         0         0         0
1    0    0  1378     1036  ...      1576      1471         0         0
2    0    0  1036     1805  ...       466       466      1471         0
3    0    0  1805      457  ...      1576      1576       466      1471
4    0    0   457      219  ...      1576       466      1576       466

[5 rows x 14 columns]
Index(['BOS', 'EOS', 'kcc', 'kcc[+1]', 'kcc[+2]', 'kcc[+3]', 'kcc[-1]',
       'kcc[-2]', 'kcc[+1]t', 'kcc[+2]t', 'kcc[+3]t', 'kcc[-1]t', 'kcc[-2]t',
       'kcc[-3]t'],
      dtype='object')
NB accuracy:  0.5895462881747867


In [0]:
from sklearn.ensemble import RandomForestClassifier
# Logistic Regression
accuracy = train_model(linear_model.LogisticRegression(), train_x, train_y, valid_x, valid_y);
print("LR accuracy: ", accuracy) # 0.6129 #0.59 with less features --ohe:93%
# SVM
#accuracy = train_model(svm.SVC(), train_x, train_y, valid_x, valid_y);
#print("SVM accuracy: ", accuracy) # 0.6818m
# Random Forest
accuracy = train_model(RandomForestClassifier(), train_x, train_y, valid_x, valid_y)
print("RF accuracy: ", accuracy )# 0.8855, ohe: 93%


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR accuracy:  0.620803763655211


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF accuracy:  0.8911570050235228


### Add One-hot encoding

In [0]:
dft.columns
dft.shape

(83606, 14)

In [0]:
# one-hot-encoding -- this will create a lot of columns
from sklearn.preprocessing import OneHotEncoder
# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()
# 2. FIT
enc.fit(dft)

# 3. Transform
onehotlabels = enc.transform(dft).toarray()
onehotlabels.shape


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(83606, 11855)

In [0]:
#onehotlabels.shape #(83606, 11855)
#onehotlabels[1:3,1:30]
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(onehotlabels, dfnb['y'], test_size=0.30, random_state=1)
train_x.shape

(58524, 11855)

In [0]:
#train_x, valid_x, train_y, valid_y = model_selection.train_test_split(onehotlabels, dfnb['y'], test_size=0.30, random_state=1)

def train_model(classifier, trains, t_labels, valids, v_labels):
  # fit the training dataset on the classifier
  classifier.fit(trains, t_labels)

  # predict the labels on validation dataset
  predictions = classifier.predict(valids)

  return metrics.accuracy_score(predictions, v_labels)

# Naive Bayes
accuracy = train_model(naive_bayes.MultinomialNB(), train_x, train_y, valid_x, valid_y);
print("NB accuracy: ", accuracy) 
#with-one hot encoding: NB accuracy:  0.8913563511681684

NB accuracy:  0.8933099433856949


Ensure all the features are independence for Naive Bayes. Thus drop columns -- only uses previous 2 char, 1 char, next char

In [0]:
# Running after one-hot encoding
from sklearn.ensemble import RandomForestClassifier

# Logistic Regression
accuracy = train_model(linear_model.LogisticRegression(), train_x, train_y, valid_x, valid_y);
print("LR accuracy: ", accuracy) # 0.6129 #0.59 with less features --ohe:93.0%

# SVM -- take too long to finish for one-hot-encoding
#accuracy = train_model(svm.SVC(), train_x, train_y, valid_x, valid_y);
#print("SVM accuracy: ", accuracy) # 0.6818m

# Random Forest
accuracy = train_model(RandomForestClassifier(), train_x, train_y, valid_x, valid_y)
print("RF accuracy: ", accuracy )# 0.8855, ohe: 93.2%

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR accuracy:  0.93126544932621


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF accuracy:  0.9311857108683518
